# NiceCX v2.0 Tutorial

In this tutorial you will learn to use NiceCX, a simple data model that is part of the ndex2 NDEx Client module.
NiceCX facilitates creating and working with networks, including interfaces to NetworkX and Pandas.
This tutorial requires Python 3.6+ and the ndex2 module, see the NDEx Client Tutorial for installation instructions.

### Import Packages Required for this Tutorial

In [2]:
from ndex2.NiceCXNetwork import NiceCXNetwork
import ndex2.client as nc
import ndex2
import networkx as nx
import pandas as pd
import os

# Five ways to create NiceCX networks:

- Starting with an Empty Network
- CX Files
- NDEx Networks
- NetworkX Networks
- Pandas DataFrames

For information on **how to access nodes, edges, attributes, etc...** see the end of this notebook ([click here](#first-bullet))

## Starting with an Empty Network

**Create an empty niceCx network**

In [3]:
niceCx_creatures = NiceCXNetwork()

**Name the Network**

The _niceCx_creatures_ will now be populated with data in which each node represents a species and has a color attribute. Each edge will specify a relationship between the two species. First, we will set the name of the network:  

In [4]:
niceCx_creatures.set_name("Food Web")

**Add Nodes and Edges**

In [5]:
fox_node = niceCx_creatures.create_node(node_name='Fox')
mouse_node = niceCx_creatures.create_node(node_name='Mouse')
bird_node = niceCx_creatures.create_node(node_name='Bird')
print(fox_node)
fox_bird_edge = niceCx_creatures.create_edge(edge_source=fox_node, edge_target=bird_node, edge_interaction='interacts-with')

fox_mouse_edge = niceCx_creatures.create_edge(edge_source=fox_node, edge_target=mouse_node, edge_interaction='interacts-with')


0


The addNode and addEdge methods return the unique ID assigned to the new node or edge. In CX, IDs are always assigned in an ascending order, although they may not always be sequential In this case, the node with the name "Fox" will have an ID of 0, "Mouse" will have 1, and so on. The edge between "Fox" and "Bird" will have the ID of 0, the next will be 1, and so on. 

**Add Attributes**

The addNodeAttribute and addEdgeAttribute require the ID in the "property_of" field and the property "name" and "values" in their respective fields.

In [6]:
print(niceCx_creatures)
niceCx_creatures.add_node_attribute(property_of=fox_node, name='Color', values='Red')

niceCx_creatures.add_node_attribute(property_of=mouse_node, name='Color', values='Gray')

niceCx_creatures.add_node_attribute(property_of=bird_node, name='Color', values='Blue')

nodes: 3 
 edges: 2


In [7]:
niceCx_creatures.add_edge_attribute(property_of=fox_mouse_edge, name='Hunted', values='On the ground')

We can now print a summary of niceCX_creatures. The returned value of the getSummary() method is structured in the same format as NetworkSummary objects returned by NDEx network search methods.

In [8]:
niceCx_creatures.print_summary()

Name: Food Web
Nodes: 3
Edges: 2
Node Attributes: 3
Edge Attributes: 1



## CX Files

In [9]:
niceCx_from_cx_file = ndex2.create_nice_cx_from_file('SimpleNetwork.cx')

niceCx_from_cx_file.print_summary()

Name: Untitled
Nodes: 2
Edges: 1
Node Attributes: 0
Edge Attributes: 0



## NDEx Networks

In [10]:
niceCx_from_server = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303')

niceCx_from_server.print_summary()

Name: CoCaNet2
Nodes: 36
Edges: 37
Node Attributes: 706
Edge Attributes: 517



## NetworkX Networks

Networks can be manipulated using NetworkX facilities and then used to create a NiceCx network. Here we create a NetworkX network from scratch:

In [11]:
G = nx.Graph()
G.add_node('ABC')
G.add_node('DEF')
G.add_node('GHI')
G.add_node('JKL')
G.add_node('MNO')
G.add_node('PQR')
G.add_node('XYZ')
G.add_edges_from([('ABC','DEF'), ('DEF', 'GHI'),('GHI', 'JKL'), 
                  ('DEF', 'JKL'), ('JKL', 'MNO'), ('DEF', 'MNO'),
                 ('MNO', 'XYZ'), ('DEF', 'PQR')])

We now use NetworkX to find the shortest path between nodes ABC and MNO and then create a subgraph from that path.

In [12]:
short_path = nx.shortest_path(G,source='ABC',target="MNO")

path_subgraph = G.subgraph(short_path)

Finally, NiceCx objects are created from both *G* and *path_subgraph*.

In [13]:
G.name = 'Created from NetworkX (full)'
niceCx_full = ndex2.create_nice_cx_from_networkx(G)

G.name = 'Created from NetworkX (shortest path)'
niceCx_short = ndex2.create_nice_cx_from_networkx(path_subgraph)
                       
niceCx_full.print_summary()
print(G.edges())
print('')

niceCx_short.print_summary()
print(path_subgraph.edges())

Name: Untitled
Nodes: 7
Edges: 8
Node Attributes: 0
Edge Attributes: 0

[('ABC', 'DEF'), ('DEF', 'GHI'), ('DEF', 'JKL'), ('DEF', 'MNO'), ('DEF', 'PQR'), ('GHI', 'JKL'), ('JKL', 'MNO'), ('MNO', 'XYZ')]

Name: Untitled
Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 0

[('ABC', 'DEF'), ('DEF', 'MNO')]


## Pandas DataFrames

### 2 Column DataFrame with No Headers:

In [14]:
data = [('ABC', 'DEF'), ('DEF', 'XYZ')]

df = pd.DataFrame.from_records(data)

niceCx_df_2_column = ndex2.create_nice_cx_from_pandas(df)

niceCx_df_2_column.print_summary()

#===========================
# CONVERT BACK TO DATAFRAME
#===========================
df_2_col_from_niceCx = niceCx_df_2_column.to_pandas_dataframe()
print(df_2_col_from_niceCx)

Name: Untitled
Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 0

  source     interaction target
0    ABC  interacts-with    DEF
1    DEF  interacts-with    XYZ


### 3 Column DataFrame with No Headers:

In [15]:
data = [('ABC', 'DEF', 'interacts-with'), ('DEF', 'XYZ', 'neighbor-of')]

df = pd.DataFrame.from_records(data)

niceCx_df_3_column = ndex2.create_nice_cx_from_pandas(df)

niceCx_df_3_column.print_summary()

#===========================
# CONVERT BACK TO DATAFRAME
#===========================
df_3_col_from_niceCx = niceCx_df_3_column.to_pandas_dataframe()
print(df_3_col_from_niceCx)

Name: Untitled
Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 0

  source     interaction target
0    ABC  interacts-with    DEF
1    DEF     neighbor-of    XYZ


### 3+ Column DataFrame with Headers to Specify Attribute Columns

In [16]:
df = pd.DataFrame.from_items([('Source', ['ABC', 'DEF']),
                              ('Target', ['DEF', 'XYZ']),
                              ('Interaction', ['interacts-with', 'neighbor-of']),
                              ('EdgeProp', ['Edge property 1', 'Edge property 2'])])

niceCx_df_with_headers = ndex2.create_nice_cx_from_pandas(df, source_field='Source', target_field='Target', 
                          edge_attr=['EdgeProp'], edge_interaction='Interaction')

niceCx_df_with_headers.print_summary()

#===========================
# CONVERT BACK TO DATAFRAME
#===========================
df_headers_from_niceCx = niceCx_df_with_headers.to_pandas_dataframe()
print(df_headers_from_niceCx)

0
Name: Untitled
Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 2

  source     interaction target
0    ABC  interacts-with    DEF
1    DEF     neighbor-of    XYZ


### Exporting a Larger and More Complex Network

More complex networks can be output to a Pandas DataFrame as well. In the following example we convert the example network we loaded from the server.

In [17]:
#======================
# CONVERT TO DATAFRAME
#======================
niceCx_from_server_df = niceCx_from_server.to_pandas_dataframe()
print(niceCx_from_server_df)

       source interaction  target
0   druggable          pp      TS
1        DHFR          pp   MAPK1
2       CHEK1          pp     BLM
3      IMPDH1          pp   MTIF2
4      IMPDH1          pp    ING5
5      IMPDH1          pp    ING4
6       MAPK1          pp     NF1
7       HDAC2          pp    WEE1
8       HDAC1          pp    WEE1
9     CSNK1G1          pp  SH3GL1
10    CSNK1G1          pp    FZR1
11     MAP2K1          pp   MAPK1
12     MAP2K1          pp   RAD52
13      HDAC6          pp   XRCC3
14      HDAC6          pp   TCEA1
15     TUBA1A          pp    ING5
16     TUBA1A          pp    ING4
17     TUBA1A          pp    BRD4
18        ADA          pp    BRD4
19      TOP3A          pp  MAP3K4
20      TOP3A          pp    FZR1
21    RABGGTB          pp  PPP2CB
22    RABGGTB          pp   XRCC3
23    RABGGTB          pp  RAD23B
24    RABGGTB          pp    BRD4
25    RABGGTB          pp     BLM
26       TOP1          pp  SH3GL1
27       TOP1          pp   LLGL1
28       TOP1 

Likewise, networks can be output as a networkx graph.

In [18]:
#=====================
# CONVERT TO NETWORKX
#=====================
niceCx_from_server_to_netx = niceCx_from_server.to_networkx()

print(niceCx_from_server_to_netx)

networkx version 1.11
CoCaNet2


**To continue with the tutorial, you must edit the following cell to replace the values of the ‘my_account’ and ‘my_password’ variables with a real NDEx account name and password.**

In [22]:
my_account="enter your username here"
my_password="enter your password here"
my_server="http://public.ndexbio.org"

if my_account == 'enter your username here':
    print('Please change the username and password before proceeding')
else:
    try:
        my_ndex=nc.Ndex2(my_server, my_account, my_password)
        my_ndex.update_status()
        print("Success.  Please continue.")
    except Exception as inst:
        print("Could not access account %s with password %s" % (my_account, my_password))
        print(inst.args)

Please change the username and password before proceeding


NiceCx networks can be saved to the NDEx server by calling **upload_to()**

In [20]:
upload_message = niceCx_df_with_headers.upload_to(my_server, my_account, my_password)
print(upload_message)

Generating CX
http://public.ndexbio.org/v2/network/7b48c493-c597-11e8-aaa6-0ac135e8bacf


## Accessing nodes, edges, attributes, etc... <a class="anchor" id="first-bullet"></a>

In [9]:
# EXAMPLE NETWORK FROM NDEx

nice_cx_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303')

Loop through all **nodes** and print when we reach the RAD52 node

In [30]:
for node_id, node in nice_cx_network.get_nodes():
    if node.get('n') == 'RAD52':
        print(node)
        break

{'@id': 1114201, 'n': 'RAD52'}


Nodes and edges are indexed by id (@id from the example above).  To look up a specific node by it's _name_ you can create a reverse look up index as follows

In [20]:
#============================
# CREATE A NAME to ID LOOKUP 
#============================
node_name_lookup = {node.get('n'): node_id for node_id, node in nice_cx_network.get_nodes()}

#========================
# GET THE 'MAP2K1' NODE
#========================
map2k1_node_id = node_name_lookup.get('MAP2K1')
map2k1_node = nice_cx_network.get_node( map2k1_node_id )
print(map2k1_node)

#=============================================
# GET A NODE ATTRIBUTE (Pathway) FOR 'MAP2K1' 
#=============================================
map2k1_pathway_attribute = nice_cx_network.get_node_attribute(map2k1_node_id, 'Pathway')
print(map2k1_pathway_attribute)

{'@id': 1114200, 'n': 'MAP2K1'}
{'po': 1114200, 'n': 'Pathway', 'v': 'Signal Transduction / Growth Regulation'}


Print all the **edges** where MAP2K1 is either a source or target

In [24]:
map2k1_edges = []

for edge_id, edge in nice_cx_network.get_edges():
    if edge.get('s') == map2k1_node_id or edge.get('t') == map2k1_node_id:
        map2k1_edges.append(edge.get('@id'))

        print(edge)


{'@id': 1114251, 's': 1114200, 't': 1114208, 'i': 'pp'}
{'@id': 1114250, 's': 1114200, 't': 1114201, 'i': 'pp'}


Or, if you would like to see the node labels represented in these edges you can look up the node and print it's name

In [32]:
for edge_id, edge in nice_cx_network.get_edges():
    if edge.get('s') == map2k1_node_id or edge.get('t') == map2k1_node_id:
        source_node = nice_cx_network.get_node( edge.get('s') )
        target_node = nice_cx_network.get_node( edge.get('t') )
        print('Source: %s Target: %s Interaction: %s' % (source_node.get('n'), target_node.get('n'), edge.get('i')))

Source: MAP2K1 Target: MAPK1 Interaction: pp
Source: MAP2K1 Target: RAD52 Interaction: pp


Print the **edge attributes** (yeastSscore) for MAP2K1 edges

In [28]:
for edge_id in map2k1_edges:
    print(nice_cx_network.get_edge_attribute(edge_id, 'yeastSscore'))


{'po': 1114251, 'n': 'yeastSscore', 'v': '-4.896', 'd': 'double'}
{'po': 1114250, 'n': 'yeastSscore', 'v': '-4.649', 'd': 'double'}
